In [263]:
import requests
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Boolean, SmallInteger, CheckConstraint
from sqlalchemy.orm import sessionmaker


In [264]:
# URL de l'API open data de la SNCF pour les objets perdus
url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=&rows=10000&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&refine.date=2016%2F01"


In [265]:
response = requests.get(url)
response.json()


{'nhits': 7243,
 'parameters': {'dataset': 'objets-trouves-restitution',
  'rows': 10000,
  'start': 0,
  'sort': ['date'],
  'facet': ['date',
   'gc_obo_date_heure_restitution_c',
   'gc_obo_gare_origine_r_name',
   'gc_obo_nature_c',
   'gc_obo_type_c',
   'gc_obo_nom_recordtype_sc_c'],
  'format': 'json',
  'timezone': 'UTC'},
 'records': [{'datasetid': 'objets-trouves-restitution',
   'recordid': '46aa8bd680d64ff47521f2f9565b3af08101f777',
   'fields': {'gc_obo_gare_origine_r_code_uic_c': '0087271007',
    'gc_obo_type_c': "Pièces d'identités et papiers personnels",
    'gc_obo_gare_origine_r_name': 'Paris Gare du Nord',
    'gc_obo_nature_c': "Carte d'identité, passeport, permis de conduire",
    'gc_obo_nom_recordtype_sc_c': 'Objet trouvé',
    'date': '2016-01-31T21:49:49+00:00'},
   'record_timestamp': '2022-12-20T10:00:48.699Z'},
  {'datasetid': 'objets-trouves-restitution',
   'recordid': '1429134dd10f77c7c671483321efee8b13b0a566',
   'fields': {'gc_obo_gare_origine_r_code_u

In [266]:
engine = create_engine('sqlite:///sncf_table.sqlite')
Base = declarative_base()


class Lost_Item(Base):
    __tablename__ = "Lost_Item"

    id = Column(String, primary_key=True, nullable=False)
    date = Column(String)
    type_objets = Column(String)
    gare= Column(String)
    code_uic = Column(String)
    date_heure_restitution = Column(String)
    
    
Base.metadata.create_all(engine)

In [268]:
engine = create_engine('sqlite:///sncf_table.sqlite')
Base = declarative_base()


Session = sessionmaker(bind=engine)
session = Session()

for itemlost in response.json()["records"]:
    try:
    
        date = itemlost['fields']['date']
        type_objets = itemlost['fields']['gc_obo_type_c']
        gare = itemlost['fields']['gc_obo_gare_origine_r_name']
        code_uic = itemlost['fields']['gc_obo_gare_origine_r_code_uic_c']
        date_heure_restitution = itemlost['fields']['gc_obo_date_heure_restitution_c']
    except KeyError:
        # KeyError will be raised if any of the keys are not present in the dictionary
        # You can handle the exception here by setting default values for the variables
        # or by raising a different exception, or by performing some other action
        date = None
        type_objets = None
        gare = None
        code_uic = None
        date_heure_restitution = None
    session.add(Lost_Item(id=itemlost['recordid'],
                        date=date,
                        type_objets=type_objets,
                        gare=gare,
                        code_uic=code_uic,
                        date_heure_restitution=date_heure_restitution))


    
    

session.commit()